## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [ ]:
reader = PdfReader("me/Profile.pdf")  # provide resources for the agent for context
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
29267 Arroyo Dr
9493168073 (Mobile)
qianyingwong@gmail.com
www.linkedin.com/in/qywong03
(LinkedIn)
Top Skills
Object-Oriented Programming
(OOP)
REST APIs
Tkinter
Languages
Chinese (Native or Bilingual)
Malay (Limited Working)
English (Native or Bilingual)
Qianying Wong
AI Intern @ AMD | Cognitive Robotics Lab | Spatial Neuroscience
Lab
Irvine, California, United States
Summary
Hi all, Ying here! 
I'm entering my fourth and final year at UC Irvine as a Cognitive
Science and Computer Science student. My unique academic
expertise has helped me gain experience in machine learning,
neuroscience research, software engineering, robotics, and game
development.
Currently, I am an intern at Advanced Micro Devices (AMD)
developing AI Agents to boost productivity and automate engineering
workflows. 
Outside of school and work, I run a lion dance team in Orange
County. Culture is important to me.
+ ‍  +   = Ying
Experience
AMD
Artificial Intelligence Intern
June 2025 - Present (2 months

In [4]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [5]:
name = "Qianying Wong"

In [6]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

# system prompt is what you're telling the model as background context
# important to tell the model if they don't know the answer, they should say so to prevent hallucination


In [7]:
system_prompt

"You are acting as Qianying Wong. You are answering questions on Qianying Wong's website, particularly questions related to Qianying Wong's career, background, skills and experience. Your responsibility is to represent Qianying Wong for interactions on the website as faithfully as possible. You are given a summary of Qianying Wong's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nHi all, Ying here! \n\nI'm entering my fourth and final year at UC Irvine as a Cognitive Science and Computer Science student. My unique academic expertise has helped me gain experience in machine learning, neuroscience research, software engineering, robotics, and game development.\n\nCurrently, I am an intern at Advanced Micro Devices (AMD) developing AI Agents to boost productivity and automate engineering workflows. \n

In [8]:
# Defines the chat function that will be used by Gradio
def chat(message, history):  # history is dictionaries in list format
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    # remember that + concatenates, so everything is actually in the same list
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [17]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [ ]:
# Create a Pydantic model for the Evaluation
# Pydantic is a framework for specifying schemas
# structured output for evaluation using evaluation output optimizer

from pydantic import BaseModel

class Evaluation(BaseModel):  # Evaluation is a subclass of BaseModel
    is_acceptable: bool
    feedback: str


In [19]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [20]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [21]:
import os
openai = OpenAI(
    # api_key=os.getenv("OPENAI_API_KEY"), 
    # base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.beta.chat.completions.parse(  
        # must use parse to get the Evaluation model, cannot use create
        model="gpt-4o",
        messages=messages,
        response_format=Evaluation
        )
    return response.choices[0].message.parsed

In [23]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [24]:
reply

'I currently do not hold any patents. My focus has been on gaining experience and developing my skills in cognitive science, computer science, and related fields through internships and research projects. If you have any other questions about my work or background, feel free to ask!'

In [25]:
evaluate(reply, "do you hold a patent?", messages[:1])
# the history is passed in as the first message, so we only pass the first message

Evaluation(is_acceptable=True, feedback="The response is acceptable as it directly answers the user's question with clarity and honesty. The agent, representing Qianying Wong, appropriately communicates that no patents are held while emphasizing the current focus on gaining experience and skill development. The agent also invites further questions, maintaining an engaging and professional tone.")

In [26]:
def rerun(reply, message, history, feedback):  # if evaluation is rejected
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
            # silly example to show how you can change the system prompt based on the message (pig latin)
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable. The agent responded in Pig Latin, which is unprofessional and not appropriate for a website representing Qianying Wong to potential clients or employers. Additionally, the response lacks any relevant information or a professional tone. The response should be clear, straightforward, and in standard English to maintain the professional and engaging tone expected of a representative of Qianying Wong.
Passed evaluation - returning reply
Passed evaluation - returning reply
